There can be improvements on this class.

Future repeaters might, for instance, support extra qubits for quantum error correction.

In [1]:
%%writefile repeaterHardware.py
%load_ext autoreload
%autoreload 2

from qutip import *

from ./ import qubit

class repeaterHardware(object):
    def __init__(id, noOfQubits, parentRepeater, globalState, opticalFibers):
        self.id = id
        self.parentRepeater = parentRepeater
#         self.globalState = self.parentRepeater  
#                                .parentRepeaterChain
#                                .globalState # this is a central globalState object passed down from the Network layer.
#         # the order of the qubits in these lists is the same as the order
#         # of the links in the link lists in the repeater class.
        global globalState
        self.globalState = globalState
        self.rightQubit = [qubit(self.id, self)]
        self.leftQubit = [qubit(self.id + 1, self)]
#         self.opticalFiber = new opticalFiber(length, )  # No, you don't create an optical fiber here,
                                                          # you create it in an upper layer --- perhaps 
                                                          # at the simulation layer: you create the needed
                                                          # hardware of your quantum internet before using it.
        self.opticalFibers = opticalFibers             # this is a list that contains the opticalFiber objects that 
                                                       # that are connected to the repeater hardware. In this simple 
                                                       # proof of concept network we only have two fibers connected 
                                                       # to any given repeater, but in future, more complicated networks
                                                       # there might be fibers going to heralding stations and so on.
                                                       
                                                          
#         self.memoryQubits = []

    def sendMessage(self, obj, msg):
        obj.handleMessage(msg)

    def handleMessage(self, msg):
        msg = msg.split('-')
        # id of the sender
        id = msg[0]
        if msg[1] === "decohered":
            # notify the link layer
            msg2 = packLinkExpired(#specify which link expired#)
            self.sendMessage(self.parentRepeater, msg2)

    def swapEntanglement(self): 
        # proper quantum gates will be performed here.
        # find the positions of the qubits in the globalState state
        # apply the right qutip gates. Assume ideal gates at this point while you're
        # building the thing.
        # Do stuff to the global state container's state.
        CNOT = cnot(N=globalState.N, control=self.leftQubit[0].id, target=self.rightQubit[0].id)
        newState = CNOT * self.globalState.state * CNOT.dag()
        Z180 = rz(180, N=self.globalState.N, target=self.leftQubit[0].id)
        Y90  = ry(90, N=self.globalState.N, target=self.leftQubit[0].id)
        H = Y90 * Z180
        newState = H * newState * H.dag()
        self.globalState.updateState(newState)
        measurement_result1 = self.measure(self.leftQubit[0])
        measurement_result2 = self.measure(self.rightQubit[0])     
        # notify the parent repeater so that it can send the classical data to
        # the other repeater.
        msg = [measurement_result1, measurement_result2]
        self.sendMessage(self.parentRepeater, msg)
        # Now the parent repeater should notify the remote repeater
        # that swapping is done and should give it the measurement results.

    def measure(self, qubit, basis):
        # bla bla
        measurement_result = ...
        # update globalState
        newState = ...
        self.globalState.updateState(newState)
        # return the measurement result
        return measurement_result
        
                
        # reset qubits
        
        # pack message with result
#         fidelity = ...
#         msg = packSwapResult("success", fidelity) # should be of the form "swap-success-fidelity-0.5".
#         # send message to parent repeater (link layer)
#         self.sendMessage(self.parentRepeater, msg)
                
    def loadQubitOnPhoton(self, qubit, photon):  # both qubit and photon are qubit objects
        # swaps the state of the photon and the local qubit 
        # (the photon should be initialized to |0>. The initialization 
        # can be noisy).
        SWAP = swap(N=self.globalState.N, targets=[qubit.id, photon.id])
        newState = SWAP * self.globalState.state * SWAP.dag()
        self.globalState.updateState(newState)

    def unloadQubitFromPhoton(self, qubit, photon):
        # swaps the state of the photon and the local qubit 
        # (the local qubit should be initialized to |0>. The initialization 
        # can be noisy). 
        SWAP = swap(N=self.globalState.N, targets=[qubit.id, photon.id])
        newState = SWAP * self.globalState.state * SWAP.dag()
        self.globalState.updateState(newState)
                
    def sendPhoton(self, photon, opticalFiber):
        opticalFiber.carryPhoton(photon)
                
    def receivePhoton(self, photon):
        # This function is called by an optical fiber to
        # alert the repeaterHardware to receive the incoming photon.
        # The repeaterHardware chooses a (physical) qubit on which to unload the 
        # qubit carried on the photon.
        self.unloadQubitFromPhoton(qubit, photon)

    def attemptLinkCreation(self):
        # here the physical details of link creation will be implemented:
        # support for heralding stations and photon transmission, etc.
        if success:
            fidelity = ... 
            msg = packLinkCreationSuccess("success", fidelity)
            self.sendMessage(self.parentRepeater, msg)
        pass

    def attemptDistillation(self):
        # apply gates on the qubits here
        pass
    

Writing repeaterHardware.py
